# SQLite Customers CRUD (Simple)

This notebook creates a local SQLite database file (`customers.db`), a `customers` table, and demonstrates basic CRUD operations.

In [1]:
# 1) Initialize and Connect to SQLite Database
import sqlite3
from pathlib import Path

DB_PATH = Path("customers.db")

# Connect to database file (creates if it doesn't exist)
conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

# Enable foreign keys just in case (not needed here but good habit)
cur.execute("PRAGMA foreign_keys = ON;")
conn.commit()

print(f"Connected to SQLite DB at {DB_PATH.resolve()}")

Connected to SQLite DB at /home/ola/webage/tjx-eic/notebooks/customers.db


In [ ]:
# 2) Create customers Table (drop if exists, then create)
cur.execute("DROP TABLE IF EXISTS customers;")
cur.execute(
    """
    CREATE TABLE customers (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        email TEXT UNIQUE NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """
)
conn.commit()
print("Table 'customers' created.")

In [ ]:
# 3) Create (Insert) Records
rows = [
    ("Alice Johnson", "alice@example.com"),
    ("Bob Smith", "bob@example.com"),
    ("Charlie Lee", "charlie@example.com"),
]
cur.executemany("INSERT INTO customers (name, email) VALUES (?, ?);", rows)
conn.commit()
print(f"Inserted {cur.rowcount if cur.rowcount != -1 else len(rows)} rows.")

In [ ]:
# 4) Read (Query) Records - All and Filtered
print("All customers:")
for row in cur.execute("SELECT id, name, email, created_at FROM customers ORDER BY id;"):
    print(row)

# Filtered read by id (example: id = 1)
customer_id = 1
cur.execute("SELECT id, name, email, created_at FROM customers WHERE id = ?;", (customer_id,))
print("\nFiltered by id=1:")
print(cur.fetchone())

In [ ]:
# 5) Update Records (change email for id=2)
cur.execute(
    "UPDATE customers SET email = ? WHERE id = ?;",
    ("bob.new@example.com", 2),
)
conn.commit()

cur.execute("SELECT id, name, email, created_at FROM customers WHERE id = ?;", (2,))
print("Updated row with id=2:")
print(cur.fetchone())

In [ ]:
# 6) Delete Records (delete id=3)
cur.execute("DELETE FROM customers WHERE id = ?;", (3,))
conn.commit()
print("Deleted row with id=3")

In [ ]:
# 7) Verify Final State
print("Remaining customers:")
for row in cur.execute("SELECT id, name, email, created_at FROM customers ORDER BY id;"):
    print(row)

cur.execute("SELECT COUNT(*) FROM customers;")
print("Total remaining:", cur.fetchone()[0])

In [ ]:
# 8) Close the Database Connection
conn.commit()
cur.close()
conn.close()
print("Connection closed.")

# Optional cleanup (uncomment to drop the table)
# conn = sqlite3.connect(DB_PATH)
# cur = conn.cursor()
# cur.execute("DROP TABLE IF EXISTS customers;")
# conn.commit()
# cur.close()
# conn.close()